# Coder House Data Engineering

Desarrollar un script que extraiga datos de una API pública. A su vez, el alumno debe crear una tabla en
Redshift para posterior carga de los datos extraidos.

In [1]:
import psycopg2
import requests
from tqdm import tqdm
import pandas as pd
import datetime

## Elección de la API

In [2]:
def obtener_datos_desde_api(codigo_pais, codigo_indicador):
    """
    Realiza una petición a la API del Banco Mundial para obtener datos de un indicador específico de un país.
    
    Parámetros:
    - codigo_pais (str): El código ISO del país.
    - codigo_indicador (str): El código del indicador económico.
    
    Retorna:
    - list: Lista de datos obtenidos de la API, o una lista vacía si hubo un error o no se encontraron datos.
    """
    url = f"http://api.worldbank.org/v2/country/{codigo_pais}/indicator/{codigo_indicador}?format=json&date=2000:2021"
    try:
        response = requests.get(url)
        data = response.json()
        if len(data) == 2 and 'page' in data[0]:
            return data[1]  # data[1] contiene los datos reales
    except requests.RequestException as e:
        print(f"Error al obtener datos desde la API: {e}")
    return []

def transformar_datos(datos_crudos):
    """
    Transforma los datos crudos obtenidos de la API en una lista de diccionarios, preparados para ser convertidos en DataFrame.
    
    Parámetros:
    - datos_crudos (list): Lista de datos crudos obtenidos de la API.
    
    Retorna:
    - list: Lista de diccionarios con los datos transformados, adecuados para conversión a DataFrame.
    """
    if datos_crudos:
        return [
            {'Country Code': dato.get('country', {}).get('id', 'N/A'),
             'Country Name': dato.get('country', {}).get('value', 'N/A'),
             'Indicator Code': dato.get('indicator', {}).get('id', 'N/A'),
             'Year': int(dato.get('date', '1900')),
             'Value': dato.get('value', None),}
            for dato in datos_crudos if dato.get('value') is not None
        ]
    return []

# Inicializar sesión de requests
session = requests.Session()

# Indicadores y países
indicadores = ['FR.INR.RINR', 'PA.NUS.FCRF', 'NY.GDP.MKTP.CD', 'NY.GDP.PCAP.CD', 'SP.POP.TOTL', 'FP.CPI.TOTL', 'GC.DOD.TOTL.GD.ZS', 'NE.EXP.GNFS.CD', 'NE.IMP.GNFS.CD', 'FI.RES.TOTL.CD', 'DT.DOD.DECT.CD', 'GC.TAX.TOTL.GD.ZS', 'SL.UEM.TOTL.NE.ZS', 'SH.XPD.CHEX.GD.ZS', 'SE.XPD.TOTL.GD.ZS', 'EG.ELC.ACCS.ZS', 'EG.USE.PCAP.KG.OE', 'EN.ATM.CO2E.PC', 'IT.NET.USER.ZS', 'SP.DYN.CBRT.IN']
indicadores_sanitizados = [indicador.replace('.', '_') for indicador in indicadores]
paises = ['MEX']

resultados = []
for pais in tqdm(paises, desc="Procesando países"):
    for indicador in tqdm(indicadores, desc=f"Indicadores para {pais}"):
        datos_crudos = obtener_datos_desde_api(pais, indicador)
        resultados.extend(transformar_datos(datos_crudos))

# Crear DataFrame
df = pd.DataFrame(resultados)

# Reemplazar puntos por guiones bajos en los nombres de los indicadores en el DataFrame
df['Indicator Code'] = df['Indicator Code'].str.replace('.', '_')
df['Value'] = df['Value'].round(2)
print(df)

Procesando países: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.07s/it]

    Country Code Country Name  Indicator Code  Year  Value
0             MX       Mexico     FR_INR_RINR  2021   0.70
1             MX       Mexico     FR_INR_RINR  2020   1.45
2             MX       Mexico     FR_INR_RINR  2019   4.00
3             MX       Mexico     FR_INR_RINR  2018   2.70
4             MX       Mexico     FR_INR_RINR  2017   0.72
..           ...          ...             ...   ...    ...
401           MX       Mexico  SP_DYN_CBRT_IN  2004  22.35
402           MX       Mexico  SP_DYN_CBRT_IN  2003  22.80
403           MX       Mexico  SP_DYN_CBRT_IN  2002  23.26
404           MX       Mexico  SP_DYN_CBRT_IN  2001  23.72
405           MX       Mexico  SP_DYN_CBRT_IN  2000  24.16

[406 rows x 5 columns]


## Estructura de la tabla

In [3]:
def cargar_datos_a_redshift(df):
    """
    Conecta a una base de datos de Redshift, crea una tabla si no existe, 
    inserta datos desde un DataFrame de pandas y cierra la conexión.
    
    Parámetros:
        df (pandas.DataFrame): DataFrame que contiene los datos a cargar.
    
    Retorna:
        None
    """
    # Datos de configuración para la conexión
    url = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
    base_de_datos = 'data-engineer-database'
    archivo_usuario = 'C:/Users/jeshu/Videos/GitHub/CoderHouseDataEngineering/Code/user_redshift.txt'
    archivo_contraseña = 'C:/Users/jeshu/Videos/GitHub/CoderHouseDataEngineering/Code/password_redshift.txt'
    
    # Leer el usuario y la contraseña de los archivos
    with open(archivo_usuario, 'r') as f:
        usuario = f.read().strip()
    with open(archivo_contraseña, 'r') as f:
        contraseña = f.read().strip()

    # Establecer la conexión con la base de datos de Redshift
    conn = psycopg2.connect(
        host=url,
        dbname=base_de_datos,
        user=usuario,
        password=contraseña,
        port=5439
    )
    conn.autocommit = True
    cursor = conn.cursor()

    # Crear tabla si no existe
    create_table_query = """
    DROP TABLE IF EXISTS economic_data;
    CREATE TABLE economic_data (
        "Country Code" VARCHAR NOT NULL,
        "Country Name" VARCHAR NOT NULL,
        "Indicator Code" VARCHAR NOT NULL,
        "Year" INTEGER NOT NULL,
        "Value" FLOAT
    );
    """
    cursor.execute(create_table_query)

    # Insertar datos desde el DataFrame
    insert_query = """
    INSERT INTO economic_data ("Country Code", "Country Name", "Indicator Code", "Year", "Value") VALUES (%s, %s, %s, %s, %s);
    """
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Cargando datos"):
        cursor.execute(insert_query, (row['Country Code'], row['Country Name'], row['Indicator Code'], row['Year'], row['Value']))

    # Cerrar la conexión
    cursor.close()
    conn.close()

cargar_datos_a_redshift(df)

Cargando datos: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 406/406 [01:40<00:00,  4.04it/s]


## Variedad de los datos

In [4]:
def obtener_datos_de_redshift():
    """
    Establece una conexión con una base de datos de Redshift, ejecuta una consulta para 
    recuperar todos los datos de la tabla 'economic_data', convierte los datos en un 
    DataFrame de pandas y cierra la conexión.

    Retorna:
        pandas.DataFrame: DataFrame que contiene los datos recuperados de la tabla.
    """
    # Datos de configuración para la conexión
    url = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
    base_de_datos = 'data-engineer-database'
    archivo_usuario = 'C:/Users/jeshu/Videos/GitHub/CoderHouseDataEngineering/Code/user_redshift.txt'
    archivo_contraseña = 'C:/Users/jeshu/Videos/GitHub/CoderHouseDataEngineering/Code/password_redshift.txt'
    
    # Leer el usuario y la contraseña de los archivos
    with open(archivo_usuario, 'r') as f:
        usuario = f.read().strip()
    with open(archivo_contraseña, 'r') as f:
        contraseña = f.read().strip()

    # Establecer la conexión con la base de datos
    conn = psycopg2.connect(
        host=url,
        dbname=base_de_datos,
        user=usuario,
        password=contraseña,
        port=5439
    )
    cursor = conn.cursor()

    # Ejecutar la consulta
    query = "SELECT * FROM economic_data;"
    cursor.execute(query)

    # Recuperar todos los datos
    data = cursor.fetchall()

    # Convertir los datos en un DataFrame para facilitar su manejo
    df = pd.DataFrame(data, columns=[desc[0] for desc in cursor.description])

    # Cerrar la conexión
    cursor.close()
    conn.close()

    return df

df = obtener_datos_de_redshift()
print(df)

    country code country name  indicator code  year  value
0             MX       Mexico     FR_INR_RINR  2021   0.70
1             MX       Mexico     FR_INR_RINR  2020   1.45
2             MX       Mexico     FR_INR_RINR  2019   4.00
3             MX       Mexico     FR_INR_RINR  2018   2.70
4             MX       Mexico     FR_INR_RINR  2017   0.72
..           ...          ...             ...   ...    ...
401           MX       Mexico  SP_DYN_CBRT_IN  2004  22.35
402           MX       Mexico  SP_DYN_CBRT_IN  2003  22.80
403           MX       Mexico  SP_DYN_CBRT_IN  2002  23.26
404           MX       Mexico  SP_DYN_CBRT_IN  2001  23.72
405           MX       Mexico  SP_DYN_CBRT_IN  2000  24.16

[406 rows x 5 columns]
